In [73]:
import pymongo
from pymongo import MongoClient
import os
from gensim.summarization import keywords as kw
import re
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
nltk.download("stopwords")
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
nltk.download('averaged_perceptron_tagger')
#import pymorphy2
#morph = pymorphy2.MorphAnalyzer()
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayakorotkaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mayakorotkaya/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [132]:
cluster = pymongo.MongoClient("mongodb+srv://User:<password>@cluster0-llq9z.mongodb.net/test?retryWrites=true&w=majority")

In [133]:
db = cluster['experiment']

In [134]:
episodes = db['results']
heroes = db['heroes']

In [135]:
db

Database(MongoClient(host=['cluster0-shard-00-01-llq9z.mongodb.net:27017', 'cluster0-shard-00-02-llq9z.mongodb.net:27017', 'cluster0-shard-00-00-llq9z.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'experiment')

Возьмем несколько серий "друзей". Про них мы будем знать их название, номер и сезон. Посчитаем для каждой ключевые слова через textrank. Посчитаем про кого в этой серии больше всего говорят (моника чендлер росс рейчел). Создадим коллекцию герои и там покажем для каждого из героев в каких сериях про них говорят

In [166]:
for file in os.listdir('friends'):
    print(file)

Friends - 4x06 - The One With The Dirty Girl.en.txt
Friends - 2x03 - The One Where Heckles Dies.en.txt
Friends - 1x01 - The One Where Monica Gets A Roommate.en.txt


In [15]:
class FriendsEpisode:
    def __init__(self):
        self.name=""
        self.season=""
        self.numb=""
        self.text=""

def getFriendsEpisode(adr):
    path = '/Users/mayakorotkaya/Desktop/friends/' + adr
    episode=FriendsEpisode()
    adr = adr.split(' - ')
    episode.name = adr[2].split('.')[0]
    episode.season = adr[1].split('x')[0]
    episode.numb = adr[1].split('x')[1]
    with open(path, 'r') as f:
        episode.text = f.read()
    
    return episode

In [86]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [91]:
def preprocessing(text):
    text = str(text)
    text = re.sub(r'[0-9\ufeff]', r'', text)
    normal_text = ''
    for t in tokenizer.tokenize(text):
                pos = get_wordnet_pos(t)
                t = lemmatizer.lemmatize(t, pos)
                if t not in english_stopwords:
                    normal_form = t + ' '
                    normal_text += normal_form
    return normal_text

### Добавление и апдейт

In [151]:
def putFriendsEpisodeInMongo(adr):
    # Загружаем текст статьи и другие ее части.
    episode = getFriendsEpisode(adr)
    
    keywords = kw(preprocessing(episode.text)).split('\n')
    # Добавляем запись с информацией об эпизоде.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"episode_name": episode.name, "episode_text": episode.text, 
              "episode_info": {"episode_season": episode.season, "episode_numb": episode.numb, 
                              "episode_keywords": keywords}}
    episode_id = episodes.insert_one(a_text).inserted_id
    print(episode_id)
    
    #ищем для каждого из имен главных персонажей, сколько раз их имя встретилось в эпизоде
    rachel = len(re.findall("Rachel", preprocessing(episode.text)))
    monica = len(re.findall("Monica", preprocessing(episode.text)))
    ross = len(re.findall("Ross", preprocessing(episode.text)))
    chandler = len(re.findall("Chandler", preprocessing(episode.text)))
    phoebe = len(re.findall("Phoebe", preprocessing(episode.text)))
    
    hero = {rachel : 'Rachel', monica : 'Monica', ross : 'Ross', chandler : 'Chandler', phoebe : 'Phoebe'}
    
    for freq in hero:
        if heroes.count_documents({"hero_name": hero[freq]}) == 0:
            hero_id = heroes.insert_one({"hero_name": hero[freq], "max_episode_id": episode_id, 
                                              "freq": freq}).inserted_id
        else:
            inis = heroes.find({"hero_name": hero[freq]})
            hero_id = inis[0]["_id"]
            if inis[0]["freq"] > freq:
                heroes.find_one_and_update({"_id": hero_id}, {"$set": {"max_episode_id": episode_id}})
            heroes.find_one_and_update({"_id": hero_id}, {"$max": {"freq":freq}})

In [141]:
putFriendsEpisodeInMongo('Friends - 1x01 - The One Where Monica Gets A Roommate.en.txt')

5e89050355ca368959f3483c


In [152]:
putFriendsEpisodeInMongo('Friends - 4x06 - The One With The Dirty Girl.en.txt')

5e8907fd55ca368959f34843


In [157]:
putFriendsEpisodeInMongo('Friends - 2x03 - The One Where Heckles Dies.en.txt')

5e89087655ca368959f34844


### Удаление

In [167]:
name1 = input('Input a name: ')
heroes.delete_one({"hero_name": name1})

Input a name: Rachel


### Поиск

In [176]:
name2 = input('Input a name: ')
results = heroes.find({'hero_name': name2})
for i in results:
    print(i)

Input a name: Monica
{'_id': ObjectId('5e89050855ca368959f3483e'), 'hero_name': 'Monica', 'max_episode_id': ObjectId('5e8907fd55ca368959f34843'), 'freq': 99}


### Сортировка

In [189]:
res = db.heroes.find().sort( [( 'freq', 1 )] )
for r in res:
    print(r)

{'_id': ObjectId('5e89050955ca368959f34841'), 'hero_name': 'Phoebe', 'max_episode_id': ObjectId('5e89087655ca368959f34844'), 'freq': 47}
{'_id': ObjectId('5e89050955ca368959f34840'), 'hero_name': 'Chandler', 'max_episode_id': ObjectId('5e89087655ca368959f34844'), 'freq': 56}
{'_id': ObjectId('5e89050855ca368959f3483f'), 'hero_name': 'Ross', 'max_episode_id': ObjectId('5e89087655ca368959f34844'), 'freq': 69}
{'_id': ObjectId('5e89050855ca368959f3483e'), 'hero_name': 'Monica', 'max_episode_id': ObjectId('5e8907fd55ca368959f34843'), 'freq': 99}


### Агрегация

In [202]:
resu = db.heroes.aggregate([{ "$group": {"_id": "$max_episode_id", "Number_of_eps": {"$sum":1}} }])
for result in resu:
    print(result)

{'_id': ObjectId('5e8907fd55ca368959f34843'), 'Number_of_eps': 1}
{'_id': ObjectId('5e89087655ca368959f34844'), 'Number_of_eps': 3}


### Работа с текстами

In [211]:
episodename = input('Введите название эпизода: ')
for d in episodes.find({'episode_name':{'$regex':'.*'+str(episodename)+'.*', "$options":"i"}}):
    print(d)

Введите название эпизода: dirty girl
{'_id': ObjectId('5e89064655ca368959f34842'), 'episode_name': 'The One With The Dirty Girl', 'episode_text': '[Scene: Central Perk, Ross and a beautiful blonde (Cheryl) are standing outside. The rest of the gang is watching from inside.]\n\nCheryl: So, thank you for the delicious dinner.\n\nRoss: You\'re welcome for a delicious dinner.\n\n[inside]\n\nPhoebe: Hey what are you guys looking at?\n\nChandler: Ross and the most beautiful girl in the world.\n\nPhoebe: Yeah, come to papa.\n\n[Cheryl walks away and Ross walks inside. Everyone stares at him in disbelief.]\n\nRoss: I know!\n\nMonica: Probably the only time I\'ll ever say this, but did you see the ass on her?\n\nChandler: Where did you, when did you, how did you... (Joey hits the back of Chandler\'s head) How did you get a girl like that?\n\nRachel: Yeah, so what is she, like a... like a spokesmodel, or an aerobics instructor, what?\n\nRoss: Actually she\'s a paleontology doctoral candidate, sp